In [22]:
import itertools


SyntaxError: invalid syntax (<ipython-input-45-caf27de49540>, line 3)

In [45]:
# Functions
# the fixture function finds the result of the two teams
def Fixture(team1, team2, draws_allowed)
    r = rand()
    team1.points +=

SyntaxError: invalid syntax (<ipython-input-45-caf27de49540>, line 3)

In [44]:
# the groupstage function resolves all the fixtures within a group stage and stores the results inside the team class
def GroupStage(team_names_in_group):
   
    match_fixtures = itertools.combinations(team_names_in_group,2)
    for team1, team2 in match_fixtures:
        Fixture(team1, team2, draws_allowed=True)
        


SyntaxError: invalid syntax (<ipython-input-45-caf27de49540>, line 3)

In [ ]:

# the Finals function resolves the knock out stages of the finals
def Finals(group_winners)
    Fixture(T1, T2, draws_allowed=False)


SyntaxError: invalid syntax (<ipython-input-45-caf27de49540>, line 3)

In [20]:
# define Team class
class Team:
    def __init__(self, team, rank, group):
        self.team = team
        self.rank = rank
        self.group = group
        self.fixtures = []
        self.points = 0
        self.result = []
        self.goal_difference = []
        
    def __repr__(self):
        return f'<Team({self.team})>'


SyntaxError: invalid syntax (<ipython-input-45-caf27de49540>, line 3)

In [19]:

# input data
# team name, FIFA rank, group ID
# 'England', 8, 'E'

teams_list = []
# load csv
with open('teams.csv', 'r') as f:
    for line in f.readlines()[1:]:
        team, rank, group = line.strip().split(',')
        teams_list.append(Team(team, rank, group))
        
teams_list

SyntaxError: invalid syntax (<ipython-input-45-caf27de49540>, line 3)

In [43]:

# for each row in csv, make an instance of the Team class.

# isolate the teams by the group they appear in
teams_in_group = [(list(g)) for _, g in itertools.groupby(teams_list, lambda x: x.group)]
teams_in_group


SyntaxError: invalid syntax (<ipython-input-45-caf27de49540>, line 3)

In [ ]:

group_winners = GroupStage(teams_in_group)


SyntaxError: invalid syntax (<ipython-input-45-caf27de49540>, line 3)

In [ ]:


winner = Finals(group_winners)


SyntaxError: invalid syntax (<ipython-input-45-caf27de49540>, line 3)